<a href="https://colab.research.google.com/github/Baleeg-Alhilali/Bioe201Group3/blob/main/Task2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Import libraries
import torch
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# PyTorch dataset
from torchvision import datasets
import torchvision.transforms as transforms
from torch.utils.data.sampler import SubsetRandomSampler

# PyTorch model
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [2]:
# check if CUDA is available
train_on_gpu = torch.cuda.is_available()

if not train_on_gpu:
    print('CUDA is not available.  Training on CPU ...')
else:
    print('CUDA is available!  Training on GPU ...')
    ! nvidia-smi

CUDA is not available.  Training on CPU ...


In [3]:
# number of subprocesses to use for data loading
num_workers = 0
# how many samples per batch to load
batch_size = 512
# percentage of training set to use as validation
valid_size = 0.2

In [4]:
# Data transform to convert data to a tensor and apply normalization

# augment train and validation dataset with RandomHorizontalFlip and RandomRotation
train_transform = transforms.Compose([
    # TODO transform data with RandomHorizontalFlip()
    # TODO transform data with RandomRotation()
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ])

test_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ])

In [5]:
# choose the training and test datasets
train_data = datasets.CIFAR10('data', train=True,
                              download=True, transform=train_transform)
test_data = datasets.CIFAR10('data', train=False,
                             download=True, transform=test_transform)

100%|██████████| 170498071/170498071 [00:02<00:00, 71623254.90it/s]


Extracting data/cifar-10-python.tar.gz to data
Files already downloaded and verified


In [6]:
# obtain training indices that will be used for validation
num_train = len(train_data)
indices = list(range(num_train))
np.random.shuffle(indices)
split = int(np.floor(valid_size * num_train))
train_idx, valid_idx = indices[split:], indices[:split]

In [7]:
# define samplers for obtaining training and validation batches
train_sampler = SubsetRandomSampler(train_idx)
valid_sampler = SubsetRandomSampler(valid_idx)

In [8]:
# prepare data loaders (combine dataset and sampler)
train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size,
    sampler=train_sampler, num_workers=num_workers)
valid_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size,
    sampler=valid_sampler, num_workers=num_workers)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=500,
    num_workers=num_workers, drop_last = True)

In [9]:
# specify the image classes
classes = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']
classes

['airplane',
 'automobile',
 'bird',
 'cat',
 'deer',
 'dog',
 'frog',
 'horse',
 'ship',
 'truck']

In [10]:
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        num_class = 10

        self.conv1 = nn.Conv2d(3, 16, 3, padding=1)
        self.bn_1 = nn.BatchNorm2d(16)
        self.pool_1 = nn.MaxPool2d(2, 2)

        self.conv2 = nn.Conv2d(16, 32, 3, padding=1)
        self.bn_2 = nn.BatchNorm2d(32)
        self.pool_2 = nn.MaxPool2d(2, 2)

        self.conv3 = nn.Conv2d(32, 64, 3, padding=1)
        self.bn_3 = nn.BatchNorm2d(64)
        self.pool_3 = nn.MaxPool2d(2, 2)

        self.adaptive_pool = nn.AdaptiveAvgPool2d((2, 2))
        self.dropout = nn.Dropout(0.25)

        self.fc = nn.Linear(64 * 2 * 2, num_class)

    def forward(self, x):
        # store the batchsize
        batch_size = x.shape[0]

        x = self.conv1(x)
        assert list(x.shape) == [batch_size, 16, 32, 32]
        x = self.bn_1(x)
        x = F.relu(x)
        x = self.pool_1(x)
        assert list(x.shape) == [batch_size, 16, 16, 16]

        x = self.conv2(x)
        assert list(x.shape) == [batch_size, 32, 16, 16]
        x = self.bn_2(x)
        x = F.relu(x)
        x = self.pool_2(x)
        assert list(x.shape) == [batch_size, 32, 8, 8]

        x = self.conv3(x)
        assert list(x.shape) == [batch_size, 64, 8, 8]
        x = self.bn_3(x)
        x = F.relu(x)
        x = self.pool_3(x)
        assert list(x.shape) == [batch_size, 64, 4, 4]

        x = self.adaptive_pool(x)
        assert list(x.shape) == [batch_size, 64, 2, 2]

        # flatten image input
        x = x.view(batch_size, -1)
        # dropout layer
        x = self.dropout(x)
        # fully-connected layer
        x = self.fc(x)
        return x

In [11]:
# create a complete CNN
model = Net()
model

# sanity check
some_input = torch.randn(512,3,32,32) # input shape batchsize * channel * W * H
the_output = model(some_input)
print(f'the shape of input: {some_input.shape}, the shape of output: {the_output.shape}')

the shape of input: torch.Size([512, 3, 32, 32]), the shape of output: torch.Size([512, 10])


In [13]:


# specify loss function (categorical cross-entropy)
criterion = nn.CrossEntropyLoss()

# specify optimizer
optimizer = optim.Adam(model.parameters(), lr=0.001)  # lr is the learning rate


In [16]:
# number of epochs to train the model
n_epochs = 35

valid_loss_min = np.Inf # track change in validation loss

for epoch in range(1, n_epochs+1):
    # print(f'==> epoch: {epoch}...')

    # keep track of training and validation loss
    train_loss = 0.0
    valid_loss = 0.0

    ###################
    # train the model #
    ###################
    model.train()
    for data, target in train_loader:
        # move tensors to GPU if CUDA is available
        if train_on_gpu:
            data, target = data.cuda(), target.cuda()
        # clear the gradients of all optimized variables
        optimizer.zero_grad()
        # forward pass: compute predicted outputs by passing inputs to the model
        output = model(data)
        # calculate the batch loss
        loss = criterion(output, target)
        # backward pass: compute gradient of the loss with respect to model parameters
        loss.backward()
        # perform a single optimization step (parameter update)
        optimizer.step()
        # update training loss
        train_loss += loss.item()*data.size(0)

    ######################
    # validate the model #
    ######################
    model.eval()
    for data, target in valid_loader:
        # move tensors to GPU if CUDA is available
        if train_on_gpu:
            data, target = data.cuda(), target.cuda()
        # forward pass: compute predicted outputs by passing inputs to the model
        output = model(data)
        # calculate the batch loss
        loss = criterion(output, target)
        # update average validation loss
        valid_loss += loss.item()*data.size(0)

    # calculate average losses
    train_loss = train_loss/len(train_loader.sampler)
    valid_loss = valid_loss/len(valid_loader.sampler)

    # print training/validation statistics
    print('Epoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f}'.format(
        epoch, train_loss, valid_loss))

    # save model if validation loss has decreased
    if valid_loss <= valid_loss_min:
        print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(
        valid_loss_min,
        valid_loss))
        torch.save(model.state_dict(), 'model_cifar.pt')
        valid_loss_min = valid_loss

Epoch: 1 	Training Loss: 0.928714 	Validation Loss: 0.919959
Validation loss decreased (inf --> 0.919959).  Saving model ...
Epoch: 2 	Training Loss: 0.909805 	Validation Loss: 0.942743
Epoch: 3 	Training Loss: 0.900868 	Validation Loss: 0.910428
Validation loss decreased (0.919959 --> 0.910428).  Saving model ...
Epoch: 4 	Training Loss: 0.880951 	Validation Loss: 0.881008
Validation loss decreased (0.910428 --> 0.881008).  Saving model ...
Epoch: 5 	Training Loss: 0.867534 	Validation Loss: 0.868889
Validation loss decreased (0.881008 --> 0.868889).  Saving model ...
Epoch: 6 	Training Loss: 0.861988 	Validation Loss: 0.849435
Validation loss decreased (0.868889 --> 0.849435).  Saving model ...
Epoch: 7 	Training Loss: 0.844179 	Validation Loss: 0.861406
Epoch: 8 	Training Loss: 0.836806 	Validation Loss: 0.907886


KeyboardInterrupt: 

In [17]:
# Load the Model with the Lowest Validation Loss
model.load_state_dict(torch.load('model_cifar.pt'))
# track test loss
test_loss = 0.0
class_correct = list(0. for i in range(10))
class_total = list(0. for i in range(10))

model.eval()
# iterate over test data
for data, target in test_loader:
    # move tensors to GPU if CUDA is available
    if train_on_gpu:
        data, target = data.cuda(), target.cuda()
    # forward pass: compute predicted outputs by passing inputs to the model
    output = model(data)
    # calculate the batch loss
    loss = criterion(output, target)
    # update test loss
    test_loss += loss.item()*data.size(0)
    # convert output probabilities to predicted class
    _, pred = torch.max(output, 1)
    # compare predictions to true label
    correct_tensor = pred.eq(target.data.view_as(pred))
    correct = np.squeeze(correct_tensor.numpy()) if not train_on_gpu else np.squeeze(correct_tensor.cpu().numpy())
    # calculate test accuracy for each object class
    for i in range(test_loader.batch_size):
        # print(len(target.data))
        label = target.data[i]
        class_correct[label] += correct[i].item()
        class_total[label] += 1

# average test loss
test_loss = test_loss/len(test_loader.dataset)
print('Test Loss: {:.6f}\n'.format(test_loss))

for i in range(10):
    if class_total[i] > 0:
        print('Test Accuracy of %5s: %2d%% (%2d/%2d)' % (
            classes[i], 100 * class_correct[i] / class_total[i],
            np.sum(class_correct[i]), np.sum(class_total[i])))
    else:
        print('Test Accuracy of %5s: N/A (no training examples)' % (classes[i]))

print('\nTest Accuracy (Overall): %2d%% (%2d/%2d)' % (
    100. * np.sum(class_correct) / np.sum(class_total),
    np.sum(class_correct), np.sum(class_total)))

Test Loss: 0.842796

Test Accuracy of airplane: 81% (810/1000)
Test Accuracy of automobile: 81% (817/1000)
Test Accuracy of  bird: 61% (615/1000)
Test Accuracy of   cat: 45% (458/1000)
Test Accuracy of  deer: 66% (662/1000)
Test Accuracy of   dog: 57% (578/1000)
Test Accuracy of  frog: 75% (758/1000)
Test Accuracy of horse: 77% (771/1000)
Test Accuracy of  ship: 80% (805/1000)
Test Accuracy of truck: 79% (792/1000)

Test Accuracy (Overall): 70% (7066/10000)
